In [41]:
#メモリのリセット
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])?  y


In [ ]:
#懸念点
#在庫推移9月4日からになっている
#在庫推移データ欠損値がある品番などがある
#かんばん毎にループ回すと時間かかる、手配運用情報の当てはめは日毎に丸め込んだ後がいい

In [42]:
#ライブラリのimport
import pandas as pd
import numpy as np
%matplotlib inline
#from matplotlib import pyplot as pyp
import matplotlib.pyplot as plt

In [43]:
# Try reading the file with 'Shift_JIS' encoding
df_shiftjis_1 = pd.read_csv('20231107使用_在庫推移9月.csv', encoding='Shift_JIS')
# Load the new CSV file into a DataFrame with 'Shift_JIS' encoding
df_shiftjis_2 = pd.read_csv('在庫推移8月4週目.csv', encoding='Shift_JIS')
# Combine the old and new data
df_shiftjis = pd.concat([df_shiftjis_1, df_shiftjis_2], ignore_index=True, sort=False)

# 集計したデータをCSVファイルに保存
with open('20231107使用_在庫推移9月（8月4週目含む）.csv', mode='w',newline='', encoding='shift_jis',errors='ignore') as f:
    df_shiftjis.to_csv(f)

In [44]:
#ファイル名の決定
file_path_LT = '20231117使用_所在管理LT_回収日8月第4週9月.csv'
file_path_tehaisuu = '20231107使用__202309_手配数_1Y_12次.csv'
file_path_tehaiunyou = '20231107使用_XR10_手配運用情報2300911_1Y（回収引当適用）.csv'
file_path_zaiko = '20231107使用_在庫推移9月（8月4週目含む）.csv'
target_month = 9

In [46]:
#データ準備
#元データの作り方
#所在管理MBからデータを吸い出す。'shift_jis'
#吸い出したデータを回収日をフィルタリングして新しいデータを作成する
pre_zaikodata = pd.read_csv(file_path_LT,encoding='shift_jis')
pre_zaikodata.head(5)

,かんばんシリアル,伝票番号,拠点所番地,品番,品名,収容数,仕入先名,仕入先工場名,ステータス,納入日,...,回収日時,発注取消日時,発注〜印刷LT,発注〜検収LT,発注〜順立装置入庫LT,発注〜順立装置出庫LT,発注〜組立LT,発注〜回収LT,更新日時,長期滞留フラグ
0,1Z23J00091508,XCGP130,20488,G1144ECE010,"ﾀｰﾐﾅﾙ,ﾓｰﾀｰｹｰﾌﾞﾙ",8,矢崎総業（株）,< NULL >,回収済,2023/9/29,...,2023/9/30 7:13,< NULL >,0.14,2.31,2.59,2.75,< NULL >,2.86,2023/9/30 7:13,< NULL >
1,1Z23J00068574,XCGL830,20488,G1144ECE010,"ﾀｰﾐﾅﾙ,ﾓｰﾀｰｹｰﾌﾞﾙ",8,矢崎総業（株）,< NULL >,回収済,2023/9/28,...,2023/9/30 7:13,< NULL >,0.15,1.99,2.23,3.79,< NULL >,3.86,2023/9/30 7:13,< NULL >
2,1Z23J00066432,XCGL630,30357,35847ECE010,ﾁｭｰﾌﾞT/Aﾙｰﾌﾞｱﾌﾟﾗｲ,40,（株）メタルテック,< NULL >,回収済,2023/9/28,...,2023/9/30 7:13,< NULL >,0.08,2.02,2.15,3.86,< NULL >,4.02,2023/9/30 7:13,< NULL >
3,1Z23J00065428,XCGL830,20488,G1144ECE010,"ﾀｰﾐﾅﾙ,ﾓｰﾀｰｹｰﾌﾞﾙ",8,矢崎総業（株）,< NULL >,回収済,2023/9/28,...,2023/9/30 7:13,< NULL >,0.15,1.99,2.23,3.76,< NULL >,3.86,2023/9/30 7:13,< NULL >
4,1Z23J00063618,XCGJ670,30358,35847ECE020,ﾁｭｰﾌﾞT/Aﾙｰﾌﾞｱﾌﾟﾗｲ,60,（株）メタルテック,< NULL >,回収済,2023/9/27,...,2023/9/30 7:13,< NULL >,2.07,3.99,4.28,6.86,< NULL >,7,2023/9/30 7:13,< NULL >


In [47]:
#各LTの計算

long = pre_zaikodata.iloc[:,1]
count = 0
for i in range(len(long)):
    if i % 5000==0:
            print(i)
    if ((pre_zaikodata.loc[i, '発注〜印刷LT'])!='< NULL >') and ((pre_zaikodata.loc[i, '発注〜検収LT'])!='< NULL >') and ((pre_zaikodata.loc[i, '発注〜順立装置入庫LT'])!='< NULL >')and ((pre_zaikodata.loc[i, '発注〜順立装置出庫LT'])!='< NULL >')and ((pre_zaikodata.loc[i, '発注〜回収LT'])!='< NULL >'):
        LT2 = float(pre_zaikodata.loc[i, '発注〜検収LT']) - float(pre_zaikodata.loc[i, '発注〜印刷LT'])
        LT3 = float(pre_zaikodata.loc[i, '発注〜順立装置入庫LT'])-float(pre_zaikodata.loc[i, '発注〜検収LT'])
        LT4 = float(pre_zaikodata.loc[i, '発注〜順立装置出庫LT'])-float(pre_zaikodata.loc[i, '発注〜順立装置入庫LT'])
        LT5 = float(pre_zaikodata.loc[i, '発注〜回収LT'])-float(pre_zaikodata.loc[i, '発注〜順立装置出庫LT'])
        LT6 = float(pre_zaikodata.loc[i, '発注〜回収LT'])-float(pre_zaikodata.loc[i, '発注〜検収LT'])
        pre_zaikodata.loc[i, '印刷検収LT'] = LT2
        pre_zaikodata.loc[i, '検収入庫LT'] = LT3
        pre_zaikodata.loc[i, '入庫出庫LT'] = LT4
        pre_zaikodata.loc[i, '出庫回収LT'] = LT5
        pre_zaikodata.loc[i, '社内LT（検収回収LT）'] = LT6
        count = count + 1
print(count)
pre_zaikodata.head(5)

0
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
55263


,かんばんシリアル,伝票番号,拠点所番地,品番,品名,収容数,仕入先名,仕入先工場名,ステータス,納入日,...,発注〜順立装置出庫LT,発注〜組立LT,発注〜回収LT,更新日時,長期滞留フラグ,印刷検収LT,検収入庫LT,入庫出庫LT,出庫回収LT,社内LT（検収回収LT）
0,1Z23J00091508,XCGP130,20488,G1144ECE010,"ﾀｰﾐﾅﾙ,ﾓｰﾀｰｹｰﾌﾞﾙ",8,矢崎総業（株）,< NULL >,回収済,2023/9/29,...,2.75,< NULL >,2.86,2023/9/30 7:13,< NULL >,2.17,0.28,0.16,0.11,0.55
1,1Z23J00068574,XCGL830,20488,G1144ECE010,"ﾀｰﾐﾅﾙ,ﾓｰﾀｰｹｰﾌﾞﾙ",8,矢崎総業（株）,< NULL >,回収済,2023/9/28,...,3.79,< NULL >,3.86,2023/9/30 7:13,< NULL >,1.84,0.24,1.56,0.07,1.87
2,1Z23J00066432,XCGL630,30357,35847ECE010,ﾁｭｰﾌﾞT/Aﾙｰﾌﾞｱﾌﾟﾗｲ,40,（株）メタルテック,< NULL >,回収済,2023/9/28,...,3.86,< NULL >,4.02,2023/9/30 7:13,< NULL >,1.94,0.13,1.71,0.16,2.00
3,1Z23J00065428,XCGL830,20488,G1144ECE010,"ﾀｰﾐﾅﾙ,ﾓｰﾀｰｹｰﾌﾞﾙ",8,矢崎総業（株）,< NULL >,回収済,2023/9/28,...,3.76,< NULL >,3.86,2023/9/30 7:13,< NULL >,1.84,0.24,1.53,0.10,1.87
4,1Z23J00063618,XCGJ670,30358,35847ECE020,ﾁｭｰﾌﾞT/Aﾙｰﾌﾞｱﾌﾟﾗｲ,60,（株）メタルテック,< NULL >,回収済,2023/9/27,...,6.86,< NULL >,7,2023/9/30 7:13,< NULL >,1.92,0.29,2.58,0.14,3.01


In [139]:
#基準在庫枚数を日数に変換する
#当月必要数でいい？

#datetime型に変換する
pre_zaikodata['回収日時'] = pd.to_datetime(pre_zaikodata['回収日時'], errors='coerce')
pre_zaikodata['納入日'] = pd.to_datetime(pre_zaikodata['納入日'], errors='coerce')

count = 0
tehaidata = pd.read_csv(file_path_tehaisuu,encoding='cp932')
long2 = tehaidata.iloc[:,1]

for i in range(len(long)):
    #print(pre_zaikodata.loc[i, '回収日時'],pre_zaikodata.loc[i,'品番'] )
    if pre_zaikodata.loc[i, '回収日時'].month == target_month:
        #print(pre_zaikodata.loc[i, '納入日（月）'].month)
        if ((pre_zaikodata.loc[i, '発注〜印刷LT'])!='< NULL >') and ((pre_zaikodata.loc[i, '発注〜検収LT'])!='< NULL >') and ((pre_zaikodata.loc[i, '発注〜順立装置入庫LT'])!='< NULL >')and ((pre_zaikodata.loc[i, '発注〜順立装置出庫LT'])!='< NULL >')and ((pre_zaikodata.loc[i, '発注〜回収LT'])!='< NULL >'):
            hinban = pre_zaikodata.loc[i,'品番'] 
            hinban2 = hinban.replace("-", "") 
            hinban3 = hinban2.replace(" ", "") 
            if i % 5000==0:
                print(i)
            for j in range(len(long2)):
                #print(j)
                tehaihinban = tehaidata.loc[j,'品番'].replace('-', '')
                tehaihinban2 = tehaihinban.replace(' ', '')
                if hinban3 == tehaihinban2:
                    day = pre_zaikodata.loc[i,'回収日時'].day
                    index = tehaidata.columns.get_loc("翌々々月稼働日数") + day
                    pre_zaikodata.loc[i,'日量数'] = int(str(tehaidata.iloc[j,index]).replace(',', ''))
                    pre_zaikodata.loc[i,'日量数（箱数）'] = pre_zaikodata.loc[i,'日量数'] /pre_zaikodata.loc[i, '収容数']
                    if int(str(tehaidata.loc[j,'当月必要数']).replace(',', '')) != 0:
                        avenichiryo = int(str(tehaidata.loc[j,'当月必要数']).replace(',', ''))/int(tehaidata.loc[j,'当月稼働日数'])
                        pre_zaikodata.loc[i,'月平均日量数'] = avenichiryo
                        pre_zaikodata.loc[i,'月平均日量数（箱数）'] = avenichiryo//pre_zaikodata.loc[i, '収容数']
                        pre_zaikodata.loc[i,'基準在庫枚数（日数）'] = (int(tehaidata.loc[j,'基準在庫枚数'])*pre_zaikodata.loc[i, '収容数'])/avenichiryo
                    else:
                        pre_zaikodata.loc[i,'月平均日量数'] = 0
                        pre_zaikodata.loc[i,'月平均日量数（箱数）'] = 0
                        pre_zaikodata.loc[i,'基準在庫枚数（日数）'] = 0
    else:
        pre_zaikodata.loc[i,'日量数'] = 0
        pre_zaikodata.loc[i,'日量数（箱数）'] = 0
        pre_zaikodata.loc[i,'月平均日量数'] = 0
        pre_zaikodata.loc[i,'月平均日量数（箱数）'] = 0
        pre_zaikodata.loc[i,'基準在庫枚数（日数）'] = 0
print(count)
pre_zaikodata.head(5)

0
5000
15000
25000
30000
35000
40000
45000
50000
0


,かんばんシリアル,伝票番号,拠点所番地,品番,品名,収容数,仕入先名,仕入先工場名,ステータス,納入日,...,整備室,箱種類,箱サイズ,箱重量,基準在庫日数,基準在庫枚数,設計かんばん数,手配区分,基準在庫日数（基準在庫枚数込み）,在庫状態
0,1Z23J00091508,XCGP130,20488,G1144ECE010,"ﾀｰﾐﾅﾙ,ﾓｰﾀｰｹｰﾌﾞﾙ",8,矢崎総業（株）,< NULL >,回収済,2023-09-29,...,1Y,ＴＰ３３１フタアリ,0.014,4.71,0.58,2.0,2.0,手配,0.611764,正常：設計値の0.75〜1倍
1,1Z23J00068574,XCGL830,20488,G1144ECE010,"ﾀｰﾐﾅﾙ,ﾓｰﾀｰｹｰﾌﾞﾙ",8,矢崎総業（株）,< NULL >,回収済,2023-09-28,...,1Y,ＴＰ３３１フタアリ,0.014,4.71,0.58,2.0,2.0,手配,0.611764,異常：設計値の2倍以上
2,1Z23J00066432,XCGL630,30357,35847ECE010,ﾁｭｰﾌﾞT/Aﾙｰﾌﾞｱﾌﾟﾗｲ,40,（株）メタルテック,< NULL >,回収済,2023-09-28,...,1Y,ＴＰ３３２フタナシ,0.022,3.30,0.50,1.0,1.0,手配,0.579410,異常：設計値の2倍以上
3,1Z23J00065428,XCGL830,20488,G1144ECE010,"ﾀｰﾐﾅﾙ,ﾓｰﾀｰｹｰﾌﾞﾙ",8,矢崎総業（株）,< NULL >,回収済,2023-09-28,...,1Y,ＴＰ３３１フタアリ,0.014,4.71,0.58,2.0,2.0,手配,0.611764,異常：設計値の2倍以上
4,1Z23J00063618,XCGJ670,30358,35847ECE020,ﾁｭｰﾌﾞT/Aﾙｰﾌﾞｱﾌﾟﾗｲ,60,（株）メタルテック,< NULL >,回収済,2023-09-27,...,1Y,ＴＰ３３２フタナシ,0.022,3.50,0.49,1.0,1.0,手配,0.609115,異常：設計値の2倍以上


In [140]:
count = 0
tehaidata = pd.read_csv(file_path_tehaiunyou,encoding='cp932')
long2 = tehaidata.iloc[:,1]

for i in range(len(long)):
    if (not pd.isnull(pre_zaikodata.loc[i, '発注〜印刷LT'])) and (not pd.isnull(pre_zaikodata.loc[i, '発注〜検収LT'])) and (not pd.isnull(pre_zaikodata.loc[i, '発注〜順立装置入庫LT']))and (not pd.isnull(pre_zaikodata.loc[i, '発注〜順立装置出庫LT']))and (not pd.isnull(pre_zaikodata.loc[i, '発注〜回収LT'])):
        hinban = pre_zaikodata.loc[i,'品番'] 
        hinban2 = hinban.replace("-", "") 
        hinban3 = hinban2.replace(" ", "") 
        if i % 5000==0:
            print(i,hinban2)
        for j in range(len(long2)):
            tehaihinban = tehaidata.loc[j,'品番'].replace('-', '')
            tehaihinban2 = tehaihinban.replace(' ', '')
            #print(tehaihinban)
            if hinban3 == tehaihinban2:
                pre_zaikodata.loc[i, 'A'] = tehaidata.loc[j,'納入ｻｲｸﾙ間隔']
                pre_zaikodata.loc[i, 'B'] = tehaidata.loc[j,'納入ｻｲｸﾙ回数']
                pre_zaikodata.loc[i, 'C'] = tehaidata.loc[j,'納入ｻｲｸﾙ情報']
                pre_zaikodata.loc[i, '便Ave'] = pre_zaikodata.loc[i,'日量数（箱数）']/pre_zaikodata.loc[i, 'B']
                pre_zaikodata.loc[i, '整備室'] = tehaidata.loc[j,'整備室']
                pre_zaikodata.loc[i, '箱種類'] = tehaidata.loc[j,'登録箱種']
                pre_zaikodata.loc[i, '箱サイズ'] = tehaidata.loc[j,'容積(m3)']
                pre_zaikodata.loc[i, '箱重量'] = tehaidata.loc[j,'総重量(Kg)']
                pre_zaikodata.loc[i, '基準在庫日数'] = tehaidata.loc[j,'基準在庫日数']
                pre_zaikodata.loc[i, '基準在庫枚数'] = tehaidata.loc[j,'基準在庫枚数']
                pre_zaikodata.loc[i, '設計かんばん数'] = pre_zaikodata.loc[i,'日量数（箱数）'] + pre_zaikodata.loc[i, '基準在庫枚数']
                pre_zaikodata.loc[i, '手配区分'] = tehaidata.loc[j,'手配区分']
                MAX= tehaidata.loc[j,'基準在庫日数'] + float(pre_zaikodata.loc[i,'基準在庫枚数（日数）'])
                pre_zaikodata.loc[i,'基準在庫日数（基準在庫枚数込み）'] = MAX
                zisseki = pre_zaikodata.loc[i, '社内LT（検収回収LT）'] 
                if  (zisseki > MAX*0)and(zisseki < 0.25*MAX):
                    pre_zaikodata.loc[i, '在庫状態'] = '異常？：設計値の0〜0.25倍'
                elif  (zisseki > MAX*0.25)and(zisseki < 0.5*MAX):
                    pre_zaikodata.loc[i, '在庫状態'] = '異常？：設計値の0.25〜0.5倍'
                elif  (zisseki > MAX*0.5)and(zisseki < 0.75*MAX):
                    pre_zaikodata.loc[i, '在庫状態'] = '正常よりの異常？：設計値の0.5〜0.75倍'
                elif  (zisseki > MAX*0.75)and(zisseki < MAX):
                    pre_zaikodata.loc[i, '在庫状態'] = '正常：設計値の0.75〜1倍'
                elif (zisseki >= MAX)and(zisseki < MAX*1.25):
                    pre_zaikodata.loc[i, '在庫状態'] = '正常：設計値の1〜1.25倍'
                elif (zisseki >= MAX*1.25)and(zisseki < 1.5*MAX):
                    pre_zaikodata.loc[i, '在庫状態'] = '正常よりの異常：設計値の1.25〜1.5倍'
                elif (zisseki >= MAX*1.5)and(zisseki < 2*MAX):
                    pre_zaikodata.loc[i, '在庫状態'] = '異常：設計値の1.5〜2倍'
                elif (zisseki >= MAX*2):
                    pre_zaikodata.loc[i, '在庫状態'] = '異常：設計値の2倍以上'
        #print(LT2,LT3,LT4,LT5)
        count = count + 1
print(count)
pre_zaikodata.head(5)

0 G1144ECE010    
5000 41310ECB010    
10000 G9210ECE020    
15000 33490ECB010    
20000 9056439A006    
25000 9036324A005    
30000 G1174ECB010    
35000 82821CWA030    
40000 35198ECE010    
45000 35165ECE010    
50000 3559850A010    
55000 G9210ECE020    
60000 G9210ECE020    
61603


,かんばんシリアル,伝票番号,拠点所番地,品番,品名,収容数,仕入先名,仕入先工場名,ステータス,納入日,...,整備室,箱種類,箱サイズ,箱重量,基準在庫日数,基準在庫枚数,設計かんばん数,手配区分,基準在庫日数（基準在庫枚数込み）,在庫状態
0,1Z23J00091508,XCGP130,20488,G1144ECE010,"ﾀｰﾐﾅﾙ,ﾓｰﾀｰｹｰﾌﾞﾙ",8,矢崎総業（株）,< NULL >,回収済,2023-09-29,...,1Y,ＴＰ３３１フタアリ,0.014,4.71,0.58,2.0,2.0,手配,0.611764,正常：設計値の0.75〜1倍
1,1Z23J00068574,XCGL830,20488,G1144ECE010,"ﾀｰﾐﾅﾙ,ﾓｰﾀｰｹｰﾌﾞﾙ",8,矢崎総業（株）,< NULL >,回収済,2023-09-28,...,1Y,ＴＰ３３１フタアリ,0.014,4.71,0.58,2.0,2.0,手配,0.611764,異常：設計値の2倍以上
2,1Z23J00066432,XCGL630,30357,35847ECE010,ﾁｭｰﾌﾞT/Aﾙｰﾌﾞｱﾌﾟﾗｲ,40,（株）メタルテック,< NULL >,回収済,2023-09-28,...,1Y,ＴＰ３３２フタナシ,0.022,3.30,0.50,1.0,1.0,手配,0.579410,異常：設計値の2倍以上
3,1Z23J00065428,XCGL830,20488,G1144ECE010,"ﾀｰﾐﾅﾙ,ﾓｰﾀｰｹｰﾌﾞﾙ",8,矢崎総業（株）,< NULL >,回収済,2023-09-28,...,1Y,ＴＰ３３１フタアリ,0.014,4.71,0.58,2.0,2.0,手配,0.611764,異常：設計値の2倍以上
4,1Z23J00063618,XCGJ670,30358,35847ECE020,ﾁｭｰﾌﾞT/Aﾙｰﾌﾞｱﾌﾟﾗｲ,60,（株）メタルテック,< NULL >,回収済,2023-09-27,...,1Y,ＴＰ３３２フタナシ,0.022,3.50,0.49,1.0,1.0,手配,0.609115,異常：設計値の2倍以上


In [141]:
# 集計したデータをCSVファイルに保存
with open('kanbandata.csv', mode='w',newline='', encoding='shift_jis',errors='ignore') as f:
    pre_zaikodata.to_csv(f)

In [142]:
kanbandata = pd.read_csv('kanbandata.csv',encoding='cp932')
kanbandata.head(5)

,Unnamed: 0,かんばんシリアル,伝票番号,拠点所番地,品番,品名,収容数,仕入先名,仕入先工場名,ステータス,...,整備室,箱種類,箱サイズ,箱重量,基準在庫日数,基準在庫枚数,設計かんばん数,手配区分,基準在庫日数（基準在庫枚数込み）,在庫状態
0,0,1Z23J00091508,XCGP130,20488,G1144ECE010,"ﾀｰﾐﾅﾙ,ﾓｰﾀｰｹｰﾌﾞﾙ",8,矢崎総業（株）,< NULL >,回収済,...,1Y,ＴＰ３３１フタアリ,0.014,4.71,0.58,2.0,2.0,手配,0.611764,正常：設計値の0.75～1倍
1,1,1Z23J00068574,XCGL830,20488,G1144ECE010,"ﾀｰﾐﾅﾙ,ﾓｰﾀｰｹｰﾌﾞﾙ",8,矢崎総業（株）,< NULL >,回収済,...,1Y,ＴＰ３３１フタアリ,0.014,4.71,0.58,2.0,2.0,手配,0.611764,異常：設計値の2倍以上
2,2,1Z23J00066432,XCGL630,30357,35847ECE010,ﾁｭｰﾌﾞT/Aﾙｰﾌﾞｱﾌﾟﾗｲ,40,（株）メタルテック,< NULL >,回収済,...,1Y,ＴＰ３３２フタナシ,0.022,3.30,0.50,1.0,1.0,手配,0.579410,異常：設計値の2倍以上
3,3,1Z23J00065428,XCGL830,20488,G1144ECE010,"ﾀｰﾐﾅﾙ,ﾓｰﾀｰｹｰﾌﾞﾙ",8,矢崎総業（株）,< NULL >,回収済,...,1Y,ＴＰ３３１フタアリ,0.014,4.71,0.58,2.0,2.0,手配,0.611764,異常：設計値の2倍以上
4,4,1Z23J00063618,XCGJ670,30358,35847ECE020,ﾁｭｰﾌﾞT/Aﾙｰﾌﾞｱﾌﾟﾗｲ,60,（株）メタルテック,< NULL >,回収済,...,1Y,ＴＰ３３２フタナシ,0.022,3.50,0.49,1.0,1.0,手配,0.609115,異常：設計値の2倍以上


In [143]:
# 'cp932'エンコーディングを使用してCSVファイルを読み込みます。
zaikodata = pd.read_csv(file_path_zaiko, encoding='cp932')

# '計測日時'を日時型に変換し、週番号を含む新しい列を追加します。
zaikodata['計測日時'] = pd.to_datetime(zaikodata['計測日時'], errors='coerce')
#欠損値削除
zaikodata.dropna(subset=['計測日時'], inplace=True)
zaikodata['Year'] = zaikodata['計測日時'].dt.year
zaikodata['Week_Number'] = zaikodata['計測日時'].dt.week

# '品番'と'Week_Number'でグループ化し、'在庫数（箱）', '入庫数（箱）', '出庫数（箱）'を合計します。
weekly_inventory_aggregated = zaikodata.groupby(['品番', 'Year', 'Week_Number']).agg({
   '在庫数（箱）': 'median',
   '入庫数（箱）': 'sum',
   '出庫数（箱）': 'sum'
}).reset_index()
#weekly_inventory_aggregated = zaikodata.groupby(['品番', 'Year', 'Week_Number']).agg({
   # '在庫数（箱）': 'median',
    #'入庫数（箱）': 'median',
    #'出庫数（箱）': 'median'
#}).reset_index()

# 集計したデータを新しいCSVファイルに保存します。
with open('kari1.csv', mode='w',newline='', encoding='shift_jis',errors='ignore') as f:
    weekly_inventory_aggregated.to_csv(f)

weekly_inventory_aggregated.head(10)

,品番,Year,Week_Number,在庫数（箱）,入庫数（箱）,出庫数（箱）
0,019120LC030,2023,34,10.0,0.0,0.0
1,019120LC030,2023,35,11.0,21.0,17.0
2,019120LC030,2023,36,12.0,26.0,25.0
3,019120LC030,2023,37,11.5,28.0,24.0
4,019120LC030,2023,38,11.0,27.0,25.0
5,019120LC030,2023,39,11.0,28.0,25.0
6,019120LC040,2023,34,2.0,0.0,0.0
7,019120LC040,2023,35,3.0,4.0,2.0
8,019120LC040,2023,36,3.0,3.0,4.0
9,019120LC040,2023,37,1.0,0.0,1.0


In [144]:
kanbandata.isnull().sum()

Unnamed: 0             0
かんばんシリアル               0
伝票番号                   0
拠点所番地                  0
品番                     0
品名                     0
収容数                    0
仕入先名                   0
仕入先工場名                 2
ステータス                  0
納入日                   12
納入便                    0
発注日時                   0
印刷日時                   0
検収日時                   0
順立装置入庫日時               0
順立装置出庫日時               0
組立投入日時                 0
回収日時                   0
発注取消日時                 0
発注～印刷LT                0
発注～検収LT                0
発注～順立装置入庫LT            0
発注～順立装置出庫LT            0
発注～組立LT                0
発注～回収LT                0
更新日時                   0
長期滞留フラグ                0
印刷検収LT              6340
検収入庫LT              6340
入庫出庫LT              6340
出庫回収LT              6340
社内LT（検収回収LT）        6340
日量数                 5372
日量数（箱数）             5372
月平均日量数              5372
月平均日量数（箱数）          5372
基準在庫枚数（日数）          5372
A                      0
B                      0


In [145]:
kanbandata.head(10)

,Unnamed: 0,かんばんシリアル,伝票番号,拠点所番地,品番,品名,収容数,仕入先名,仕入先工場名,ステータス,...,整備室,箱種類,箱サイズ,箱重量,基準在庫日数,基準在庫枚数,設計かんばん数,手配区分,基準在庫日数（基準在庫枚数込み）,在庫状態
0,0,1Z23J00091508,XCGP130,20488,G1144ECE010,"ﾀｰﾐﾅﾙ,ﾓｰﾀｰｹｰﾌﾞﾙ",8,矢崎総業（株）,< NULL >,回収済,...,1Y,ＴＰ３３１フタアリ,0.014,4.71,0.58,2.0,2.0,手配,0.611764,正常：設計値の0.75～1倍
1,1,1Z23J00068574,XCGL830,20488,G1144ECE010,"ﾀｰﾐﾅﾙ,ﾓｰﾀｰｹｰﾌﾞﾙ",8,矢崎総業（株）,< NULL >,回収済,...,1Y,ＴＰ３３１フタアリ,0.014,4.71,0.58,2.0,2.0,手配,0.611764,異常：設計値の2倍以上
2,2,1Z23J00066432,XCGL630,30357,35847ECE010,ﾁｭｰﾌﾞT/Aﾙｰﾌﾞｱﾌﾟﾗｲ,40,（株）メタルテック,< NULL >,回収済,...,1Y,ＴＰ３３２フタナシ,0.022,3.30,0.50,1.0,1.0,手配,0.579410,異常：設計値の2倍以上
3,3,1Z23J00065428,XCGL830,20488,G1144ECE010,"ﾀｰﾐﾅﾙ,ﾓｰﾀｰｹｰﾌﾞﾙ",8,矢崎総業（株）,< NULL >,回収済,...,1Y,ＴＰ３３１フタアリ,0.014,4.71,0.58,2.0,2.0,手配,0.611764,異常：設計値の2倍以上
4,4,1Z23J00063618,XCGJ670,30358,35847ECE020,ﾁｭｰﾌﾞT/Aﾙｰﾌﾞｱﾌﾟﾗｲ,60,（株）メタルテック,< NULL >,回収済,...,1Y,ＴＰ３３２フタナシ,0.022,3.50,0.49,1.0,1.0,手配,0.609115,異常：設計値の2倍以上
5,5,1Z23J00063158,XCGJ730,20140,35441TFA010,ｽﾄﾚｰﾅｰｵｲﾙ,100,（株）ニフコ,名古屋事業所,回収済,...,1Y,ＴＰ３３１フタナシ,0.012,0.69,0.67,1.0,1.0,手配,0.924978,異常：設計値の2倍以上
6,6,1Z23J00063158,XCGJ730,20140,35441TFA010,ｽﾄﾚｰﾅｰｵｲﾙ,100,（株）ニフコ,名古屋事業所,回収済,...,1Y,ＴＰ３３１フタナシ,0.012,0.69,0.67,1.0,1.0,手配,0.924978,異常：設計値の2倍以上
7,7,1Z23J00062502,XCGJ730,20141,35441ECB010,ｽﾄﾚｰﾅｰｵｲﾙ,48,（株）ニフコ,名古屋事業所,回収済,...,1Y,ＴＰ３３１フタナシ,0.012,0.75,0.67,3.0,3.0,手配,0.830731,異常：設計値の2倍以上
8,8,1Z23J00061320,XCGJ670,30355,35847ECB020,ﾁｭｰﾌﾞT/Aﾙｰﾌﾞｱﾌﾟﾗｲ,70,（株）メタルテック,< NULL >,回収済,...,1Y,ＴＰ３３２フタナシ,0.022,2.74,0.50,1.0,1.0,手配,0.678485,異常：設計値の2倍以上
9,9,1Z23J00055330,XCGF520,20364,9034118090,ﾌﾟﾗｸﾞｳｲｽﾞﾍﾂﾄﾞｽﾄﾚ-ﾄｽｸﾘﾕ-,300,（株）杉浦製作所,< NULL >,回収済,...,1Y,ＴＰ３３１フタナシ,0.012,10.36,0.54,3.0,3.0,手配,1.544571,正常：設計値の1～1.25倍


In [156]:
# '納入日'を日時型に変換
kanbandata['回収日時'] = pd.to_datetime(kanbandata['回収日時'], errors='coerce')
# 日時型に変換できなかった行を削除
kanbandata.dropna(subset=['回収日時'], inplace=True)
kanbandata.dropna(subset=['印刷検収LT'], inplace=True)
kanbandata.dropna(subset=['検収入庫LT'], inplace=True)
kanbandata.dropna(subset=['入庫出庫LT'], inplace=True)
kanbandata.dropna(subset=['出庫回収LT'], inplace=True)
kanbandata.dropna(subset=['社内LT（検収回収LT）'], inplace=True)
kanbandata.dropna(subset=['日量数（箱数）'], inplace=True)

# 'Week_Number' 列を追加（年と週の番号を取得）
kanbandata['Year'] = kanbandata['回収日時'].dt.year
kanbandata['Week_Number'] = kanbandata['回収日時'].dt.week
kanbandata['day'] = kanbandata['回収日時'].dt.day

weekly_aggregated_day = kanbandata.groupby(['品番', '仕入先名','Year','Week_Number', 'day',  '基準在庫日数','基準在庫日数（基準在庫枚数込み）', '基準在庫枚数','箱種類','A','B','C']).agg({'日量数（箱数）': 'mean'})
weekly_aggregated_day_sum = weekly_aggregated_day.groupby(['品番', '仕入先名','Year','Week_Number', '基準在庫日数','基準在庫日数（基準在庫枚数込み）', '基準在庫枚数','箱種類','A','B','C']).agg({'日量数（箱数）': 'sum'})

# '品番'と'Week_Number'でグループ化し、'収容数'などを合計
#納入便入れるとできない
weekly_aggregated = kanbandata.groupby(['品番', '仕入先名','Year', 'Week_Number', '基準在庫日数','基準在庫日数（基準在庫枚数込み）', '基準在庫枚数','箱種類','A','B','C']).agg({'収容数': 'median', '印刷検収LT': 'median', '検収入庫LT': 'median','入庫出庫LT': 'median','出庫回収LT': 'median','社内LT（検収回収LT）': 'median'})
# '品番'と'Week_Number'でグループ化し、各グループの行数（かんばん数）をカウント
weekly_kanban_count = kanbandata.groupby(['品番','仕入先名','Year', 'Week_Number', '基準在庫日数','基準在庫日数（基準在庫枚数込み）', '基準在庫枚数','箱種類','A','B','C']).size().reset_index(name='回収かんばん数')

# 2つの集計データセットを結合
merged_weekly_data = pd.merge(weekly_aggregated, weekly_kanban_count, on=['品番', 'Year', 'Week_Number', '基準在庫日数','基準在庫日数（基準在庫枚数込み）', '基準在庫枚数','箱種類','A','B','C'], how='outer')
merged_weekly_data = pd.merge(merged_weekly_data, weekly_aggregated_day_sum, on=['品番', 'Year', 'Week_Number', '基準在庫日数','基準在庫日数（基準在庫枚数込み）', '基準在庫枚数','箱種類','A','B','C'], how='outer')

# 集計したデータをCSVファイルに保存
with open('kari2.csv', mode='w',newline='', encoding='shift_jis',errors='ignore') as f:
    merged_weekly_data.to_csv(f)

merged_weekly_data.head(10)
#weekly_aggregated_day.head(30)
#weekly_aggregated_day_sum.head(30)

,品番,Year,Week_Number,基準在庫日数,基準在庫日数（基準在庫枚数込み）,基準在庫枚数,箱種類,A,B,C,収容数,印刷検収LT,検収入庫LT,入庫出庫LT,出庫回収LT,社内LT（検収回収LT）,仕入先名,回収かんばん数,日量数（箱数）
0,019128GA010,2023,35,1.07,1.070000,1.0,ＴＰ３４２フタナシ,1.0,2.0,3.64,50,1.850,0.350,0.985,0.230,3.110,石黒ゴム工業（株）,64,0.000000
1,019128GA010,2023,35,1.07,1.139097,1.0,ＴＰ３４２フタナシ,1.0,2.0,3.64,50,1.565,0.310,6.800,0.245,7.330,石黒ゴム工業（株）,26,17.600000
2,019128GA010,2023,36,1.07,1.139097,1.0,ＴＰ３４２フタナシ,1.0,2.0,3.64,50,1.850,0.340,2.760,0.230,3.510,石黒ゴム工業（株）,93,94.540000
3,019128GA010,2023,37,1.07,1.139097,1.0,ＴＰ３４２フタナシ,1.0,2.0,3.64,50,1.850,0.280,2.440,0.235,3.640,石黒ゴム工業（株）,90,88.580000
4,019128GA010,2023,38,1.07,1.139097,1.0,ＴＰ３４２フタナシ,1.0,2.0,3.64,50,1.850,0.290,2.660,0.250,3.510,石黒ゴム工業（株）,87,86.520000
5,019128GA010,2023,39,1.07,1.139097,1.0,ＴＰ３４２フタナシ,1.0,2.0,3.64,50,1.850,0.345,2.625,0.220,3.575,石黒ゴム工業（株）,92,89.040000
6,01912ECB010,2023,35,0.85,0.850000,1.0,ＴＰ３３２フタナシ,1.0,2.0,3.64,300,1.850,0.300,4.150,0.315,4.930,石黒ゴム工業（株）,22,0.000000
7,01912ECB010,2023,35,0.85,1.057337,1.0,ＴＰ３３２フタナシ,1.0,2.0,3.64,300,1.850,0.350,2.570,0.310,3.390,石黒ゴム工業（株）,9,5.866667
8,01912ECB010,2023,36,0.85,1.057337,1.0,ＴＰ３３２フタナシ,1.0,2.0,3.64,300,1.850,0.300,2.790,0.290,3.520,石黒ゴム工業（株）,31,31.513333
9,01912ECB010,2023,37,0.85,1.057337,1.0,ＴＰ３３２フタナシ,1.0,2.0,3.64,300,1.850,0.250,2.620,0.250,3.410,石黒ゴム工業（株）,31,29.506667


In [157]:
# 2つの集計データセットを結合
merged_data = pd.merge(weekly_inventory_aggregated, merged_weekly_data, on=['品番', 'Year', 'Week_Number'], how='inner')
# '基準在庫日数' から '社内LT（検収～回収LT）' を引いた新しい列 'LT_差分' を追加
merged_data['社内LT/設計値LT'] = merged_data['社内LT（検収回収LT）']/merged_data['基準在庫日数']
merged_data['社内LT/設計値LT（基準在庫枚数込み）'] = merged_data['社内LT（検収回収LT）']/merged_data['基準在庫日数（基準在庫枚数込み）']
#
merged_data['順立装置在庫量/設計値MIN']=merged_data['在庫数（箱）']/(0.1*(merged_data['日量数（箱数）']*merged_data['A']*(1+merged_data['C'])/merged_data['B']))

# Load the new CSV file into a DataFrame with 'Shift_JIS' encoding
pitch = pd.read_csv('不等ピッチ情報.csv', encoding='Shift_JIS')

longi = len(merged_data.loc[:,"品番"])
longj = len(pitch.loc[:,"仕入先名"])

for i in range(longi):
    for j in range(longj):
        if (merged_data.loc[i, '仕入先名'] == pitch.loc[j,'仕入先名']):
            merged_data.loc[i,"不等ピッチ係数"] = pitch.loc[j,"不等ピッチ係数日"]
            
merged_data['順立装置在庫量/設計値MAX'] = merged_data['在庫数（箱）']/(merged_data['順立装置在庫量/設計値MIN'] + merged_data['日量数（箱数）']/merged_data['B'] + merged_data['日量数（箱数）']*merged_data.loc[i,"不等ピッチ係数"])

#S = int(str(df_design.loc[j,'収容数']).replace(',', ''))
#df_merge.loc[count,'最小値'] = 0.1 * ((daily/S)  * A * (1 + C) / B) 
#df_merge.loc[count,'最大値'] = df_merge.loc[count,'最小値'] + (daily / S)/B + (daily/S)*df_design.loc[j,'不等ピッチ']

# 集計したデータをCSVファイルに保存
with open('kari5.csv', mode='w',newline='', encoding='shift_jis',errors='ignore') as f:
    merged_data.to_csv(f)

merged_data.head(300)

,品番,Year,Week_Number,在庫数（箱）,入庫数（箱）,出庫数（箱）,基準在庫日数,基準在庫日数（基準在庫枚数込み）,基準在庫枚数,箱種類,...,出庫回収LT,社内LT（検収回収LT）,仕入先名,回収かんばん数,日量数（箱数）,社内LT/設計値LT,社内LT/設計値LT（基準在庫枚数込み）,順立装置在庫量/設計値MIN,不等ピッチ係数,順立装置在庫量/設計値MAX
0,019128GA010,2023,35,49.0,78.0,65.0,1.07,1.070000,1.0,ＴＰ３４２フタナシ,...,0.230,3.11,石黒ゴム工業（株）,64,0.000000,2.906542,2.906542,inf,0.29,0.000000
1,019128GA010,2023,35,49.0,78.0,65.0,1.07,1.139097,1.0,ＴＰ３４２フタナシ,...,0.245,7.33,石黒ゴム工業（株）,26,17.600000,6.850467,6.434919,12.000392,0.29,2.260107
2,019128GA010,2023,36,41.0,87.0,85.0,1.07,1.139097,1.0,ＴＰ３４２フタナシ,...,0.230,3.51,石黒ゴム工業（株）,93,94.540000,3.280374,3.081387,1.869305,0.29,0.761144
3,019128GA010,2023,37,40.0,88.0,82.0,1.07,1.139097,1.0,ＴＰ３４２フタナシ,...,0.235,3.64,石黒ゴム工業（株）,90,88.580000,3.401869,3.195512,1.946419,0.29,0.789493
4,019128GA010,2023,38,37.0,74.0,78.0,1.07,1.139097,1.0,ＴＰ３４２フタナシ,...,0.250,3.51,石黒ゴム工業（株）,87,86.520000,3.280374,3.081387,1.843305,0.29,0.748544
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,35847ECB010,2023,37,5.0,28.0,28.0,0.51,0.713983,1.0,ＴＰ３３２フタナシ,...,0.260,1.74,（株）メタルテック,17,24.212500,3.411765,2.437034,0.712086,0.00,0.191313
296,35847ECB010,2023,38,4.0,28.0,27.0,0.51,0.713983,1.0,ＴＰ３３２フタナシ,...,0.315,2.00,（株）メタルテック,28,23.412500,3.921569,2.801189,0.589134,0.00,0.158905
297,35847ECB010,2023,39,5.0,30.0,28.0,0.51,0.713983,1.0,ＴＰ３３２フタナシ,...,0.310,1.69,（株）メタルテック,31,24.575000,3.313725,2.367005,0.701582,0.00,0.188641
298,35847ECB020,2023,35,8.0,22.0,17.0,0.50,0.500000,1.0,ＴＰ３３２フタナシ,...,0.390,2.77,（株）メタルテック,19,0.000000,5.540000,5.540000,inf,0.00,0.000000


In [158]:
# 新しいCSVファイルのパスを定義します。
#shiftjisでMBから落とす
new_file_path = '組立稼働率（実績）.csv'

# 'cp932'エンコーディングを使用してCSVファイルを読み込みます。
AVAILABLE_RATE = pd.read_csv(new_file_path, encoding='cp932')

# データフレームのインデックスを日付型に変換
AVAILABLE_RATE['LINE_DATE'] = pd.to_datetime(AVAILABLE_RATE['LINE_DATE'])
#AVAILABLE_RATE.set_index('LINE_DATE', inplace=True)
# 'WEEK_NUMBER'列を追加し、'LINE_DATE'の週番号を計算
AVAILABLE_RATE['Week_Number'] = AVAILABLE_RATE['LINE_DATE'].dt.week
AVAILABLE_RATE['WEEKDAY'] = AVAILABLE_RATE['LINE_DATE'].dt.weekday

# 土日を除外
AVAILABLE_RATE2 = AVAILABLE_RATE[(AVAILABLE_RATE['WEEKDAY'] != 5) & (AVAILABLE_RATE['WEEKDAY'] != 6)]

# 'AVAILABLE_RATE'列を週単位でリサンプリングし、平均を計算
weekly_average_available_rate = AVAILABLE_RATE2.groupby('Week_Number').agg({'OPERATION_RATE': 'median', 'KAKO_CNT': 'median'})

weekly_average_available_rate = weekly_average_available_rate.rename(columns={'OPERATION_RATE': '時間稼働率'})
weekly_average_available_rate = weekly_average_available_rate.rename(columns={'KAKO_CNT': '加工数'})

# 集計したデータを新しいCSVファイルに保存します。
with open('kari4.csv', mode='w',newline='', encoding='shift_jis',errors='ignore') as f:
    weekly_average_available_rate.to_csv(f)

#AVAILABLE_RATE.head(300)
weekly_average_available_rate.head(30)

,時間稼働率,加工数
Week_Number,,
14,97.145,409.5
15,98.095,463.0
16,95.640,449.5
17,93.025,490.5
18,0.000,0.0
19,79.925,350.0
20,94.195,506.5
21,91.335,496.0
22,90.475,506.5


In [159]:
# 2つの集計データセットを結合
merged_data = pd.merge(merged_data, weekly_average_available_rate, on=['Week_Number'], how='inner')
# 集計したデータを新しいCSVファイルに保存します。
with open('kari6.csv', mode='w',newline='', encoding='shift_jis',errors='ignore') as f:
    merged_data.to_csv(f)
merged_data.head(300)

,品番,Year,Week_Number,在庫数（箱）,入庫数（箱）,出庫数（箱）,基準在庫日数,基準在庫日数（基準在庫枚数込み）,基準在庫枚数,箱種類,...,仕入先名,回収かんばん数,日量数（箱数）,社内LT/設計値LT,社内LT/設計値LT（基準在庫枚数込み）,順立装置在庫量/設計値MIN,不等ピッチ係数,順立装置在庫量/設計値MAX,時間稼働率,加工数
0,019128GA010,2023,35,49.0,78.0,65.0,1.07,1.070000,1.0,ＴＰ３４２フタナシ,...,石黒ゴム工業（株）,64,0.000000,2.906542,2.906542,inf,0.29,0.000000,99.885,475.0
1,019128GA010,2023,35,49.0,78.0,65.0,1.07,1.139097,1.0,ＴＰ３４２フタナシ,...,石黒ゴム工業（株）,26,17.600000,6.850467,6.434919,12.000392,0.29,2.260107,99.885,475.0
2,01912ECB010,2023,35,14.0,23.0,22.0,0.85,0.850000,1.0,ＴＰ３３２フタナシ,...,石黒ゴム工業（株）,22,0.000000,5.800000,5.800000,inf,0.29,0.000000,99.885,475.0
3,01912ECB010,2023,35,14.0,23.0,22.0,0.85,1.057337,1.0,ＴＰ３３２フタナシ,...,石黒ゴム工業（株）,9,5.866667,3.988235,3.206169,10.286050,0.29,1.036061,99.885,475.0
4,01912ECB040,2023,35,33.0,71.0,65.0,0.25,0.250000,1.0,ＴＰ３４２フタナシ,...,愛産樹脂工業（株）,65,0.000000,15.080000,15.080000,inf,0.00,0.000000,99.885,475.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,G1163ECE010,2023,35,11.0,75.0,74.0,0.53,0.580879,1.0,ＴＰ３３１フタナシ,...,（株）青山製作所,28,19.140000,1.688679,1.540767,5.101000,0.08,1.189446,99.885,475.0
296,G117362010,2023,35,59.5,205.0,181.0,0.85,0.850000,1.0,ＴＰ３３１．５フタナシ,...,多摩川精機販売（株）,182,0.000000,3.647059,3.647059,inf,0.00,0.000000,99.885,475.0
297,G117362010,2023,35,59.5,205.0,181.0,0.85,0.870091,1.0,ＴＰ３３１．５フタナシ,...,多摩川精機販売（株）,70,48.888889,2.305882,2.252637,3.005051,0.00,1.094990,99.885,475.0
298,G1174ECB010,2023,35,12.5,24.0,27.0,0.85,0.850000,1.0,ＴＰ３６２フタナシ,...,住友電装（株）,26,0.000000,4.482353,4.482353,inf,0.00,0.000000,99.885,475.0


In [160]:
# 品番と週番号でデータを並べ替え
merged_data2 = merged_data.sort_values(by=['品番', 'Week_Number'])
# 同じ品番ごとに在庫数の週間差分を計算
merged_data2['先週からの順立装置在庫量増加率'] = merged_data2.groupby('品番')['在庫数（箱）'].pct_change()
merged_data2['先週からの社内LT増加率'] = merged_data2.groupby('品番')['社内LT（検収回収LT）'].pct_change()
merged_data2['先週からの回収かんばん増加率'] = merged_data2.groupby('品番')['回収かんばん数'].pct_change()
# 差分がNaN（最初の週）である場合は0に置き換え
merged_data2['先週からの順立装置在庫量増加率'].fillna(0, inplace=True)
merged_data2['先週からの社内LT増加率'].fillna(0, inplace=True)
merged_data2['先週からの回収かんばん増加率'].fillna(0, inplace=True)
merged_data2.head(30)

,品番,Year,Week_Number,在庫数（箱）,入庫数（箱）,出庫数（箱）,基準在庫日数,基準在庫日数（基準在庫枚数込み）,基準在庫枚数,箱種類,...,社内LT/設計値LT,社内LT/設計値LT（基準在庫枚数込み）,順立装置在庫量/設計値MIN,不等ピッチ係数,順立装置在庫量/設計値MAX,時間稼働率,加工数,先週からの順立装置在庫量増加率,先週からの社内LT増加率,先週からの回収かんばん増加率
0,019128GA010,2023,35,49.0,78.0,65.0,1.07,1.070000,1.0,ＴＰ３４２フタナシ,...,2.906542,2.906542,inf,0.29,0.000000,99.885,475.0,0.000000,0.000000,0.000000
1,019128GA010,2023,35,49.0,78.0,65.0,1.07,1.139097,1.0,ＴＰ３４２フタナシ,...,6.850467,6.434919,12.000392,0.29,2.260107,99.885,475.0,0.000000,1.356913,-0.593750
312,019128GA010,2023,36,41.0,87.0,85.0,1.07,1.139097,1.0,ＴＰ３４２フタナシ,...,3.280374,3.081387,1.869305,0.29,0.761144,99.895,480.0,-0.163265,-0.521146,2.576923
482,019128GA010,2023,37,40.0,88.0,82.0,1.07,1.139097,1.0,ＴＰ３４２フタナシ,...,3.401869,3.195512,1.946419,0.29,0.789493,99.440,462.0,-0.024390,0.037037,-0.032258
651,019128GA010,2023,38,37.0,74.0,78.0,1.07,1.139097,1.0,ＴＰ３４２フタナシ,...,3.280374,3.081387,1.843305,0.29,0.748544,99.905,450.0,-0.075000,-0.035714,-0.033333
811,019128GA010,2023,39,40.0,84.0,82.0,1.07,1.139097,1.0,ＴＰ３４２フタナシ,...,3.341121,3.138450,1.936363,0.29,0.785726,99.955,464.0,0.081081,0.018519,0.057471
2,01912ECB010,2023,35,14.0,23.0,22.0,0.85,0.850000,1.0,ＴＰ３３２フタナシ,...,5.800000,5.800000,inf,0.29,0.000000,99.885,475.0,0.000000,0.000000,0.000000
3,01912ECB010,2023,35,14.0,23.0,22.0,0.85,1.057337,1.0,ＴＰ３３２フタナシ,...,3.988235,3.206169,10.286050,0.29,1.036061,99.885,475.0,0.000000,-0.312373,-0.590909
313,01912ECB010,2023,36,13.0,30.0,28.0,0.85,1.057337,1.0,ＴＰ３３２フタナシ,...,4.141176,3.329120,1.778120,0.29,0.680256,99.895,480.0,-0.071429,0.038348,2.444444
483,01912ECB010,2023,37,13.0,28.0,28.0,0.85,1.057337,1.0,ＴＰ３３２フタナシ,...,4.011765,3.225085,1.899045,0.29,0.717134,99.440,462.0,0.000000,-0.031250,0.000000


In [161]:
# 新しいCSVファイルのパスを定義します。
new_file_path = '20231107使用_使用工程.csv'
# 'cp932'エンコーディングを使用してCSVファイルを読み込みます。
step = pd.read_csv(new_file_path, encoding='cp932')

longi = len(merged_data2.loc[:,"品番"])
longj = len(step.loc[:,"品番"])

for i in range(longi):
    hinban = merged_data2.loc[i,'品番'] 
    hinban2 = hinban.replace("-", "") 
    hinban3 = hinban2.replace(" ", "") 
    for j in range(longj):
        stephinban = step.loc[j,'品番'].replace('-', '')
        stephinban2 = stephinban.replace(' ', '')
        if (hinban3 == stephinban2):
            merged_data2.loc[i,"使用工程"] = step.loc[j,"使用工程"]
    
merged_data2.head(5)

,品番,Year,Week_Number,在庫数（箱）,入庫数（箱）,出庫数（箱）,基準在庫日数,基準在庫日数（基準在庫枚数込み）,基準在庫枚数,箱種類,...,社内LT/設計値LT（基準在庫枚数込み）,順立装置在庫量/設計値MIN,不等ピッチ係数,順立装置在庫量/設計値MAX,時間稼働率,加工数,先週からの順立装置在庫量増加率,先週からの社内LT増加率,先週からの回収かんばん増加率,使用工程
0,019128GA010,2023,35,49.0,78.0,65.0,1.07,1.070000,1.0,ＴＰ３４２フタナシ,...,2.906542,inf,0.29,0.000000,99.885,475.0,0.000000,0.000000,0.000000,8.ファイナル後半
1,019128GA010,2023,35,49.0,78.0,65.0,1.07,1.139097,1.0,ＴＰ３４２フタナシ,...,6.434919,12.000392,0.29,2.260107,99.885,475.0,0.000000,1.356913,-0.593750,8.ファイナル後半
312,019128GA010,2023,36,41.0,87.0,85.0,1.07,1.139097,1.0,ＴＰ３４２フタナシ,...,3.081387,1.869305,0.29,0.761144,99.895,480.0,-0.163265,-0.521146,2.576923,8.ファイナル後半
482,019128GA010,2023,37,40.0,88.0,82.0,1.07,1.139097,1.0,ＴＰ３４２フタナシ,...,3.195512,1.946419,0.29,0.789493,99.440,462.0,-0.024390,0.037037,-0.032258,8.ファイナル後半
651,019128GA010,2023,38,37.0,74.0,78.0,1.07,1.139097,1.0,ＴＰ３４２フタナシ,...,3.081387,1.843305,0.29,0.748544,99.905,450.0,-0.075000,-0.035714,-0.033333,8.ファイナル後半


In [162]:
#納入数追加
#入庫数合計
#出庫数合計

import pandas as pd

# Load the provided CSV file
file_path = '20231107使用_所在管理LT_納入日0828_0929.csv'

# Since the file is likely in Japanese, we'll use the encoding 'utf-8' or 'shift_jis'
try:
    data = pd.read_csv(file_path, encoding='utf-8')
except UnicodeDecodeError:
    data = pd.read_csv(file_path, encoding='shift_jis')

data['検収日時'] = pd.to_datetime(data['検収日時'], errors='coerce')
data['Year'] = data['検収日時'].dt.year
data['Week_Number'] = data['検収日時'].dt.week

# Grouping the data by '品番' (part number) and '納入日' (delivery date) and counting the number of Kanbans
nounyuu_kanban = data.groupby(['品番','仕入先名','Year','Week_Number']).size().reset_index(name='納入かんばん数')

# Displaying the result
#nounyuu_kanban.head(30)

merged_data3 = pd.merge(merged_data2, nounyuu_kanban, on=['品番','仕入先名','Year','Week_Number'], how='outer')
merged_data3.head(30)

,品番,Year,Week_Number,在庫数（箱）,入庫数（箱）,出庫数（箱）,基準在庫日数,基準在庫日数（基準在庫枚数込み）,基準在庫枚数,箱種類,...,順立装置在庫量/設計値MIN,不等ピッチ係数,順立装置在庫量/設計値MAX,時間稼働率,加工数,先週からの順立装置在庫量増加率,先週からの社内LT増加率,先週からの回収かんばん増加率,使用工程,納入かんばん数
0,019128GA010,2023,35,49.0,78.0,65.0,1.07,1.070000,1.0,ＴＰ３４２フタナシ,...,inf,0.29,0.000000,99.885,475.0,0.000000,0.000000,0.000000,8.ファイナル後半,92.0
1,019128GA010,2023,35,49.0,78.0,65.0,1.07,1.139097,1.0,ＴＰ３４２フタナシ,...,12.000392,0.29,2.260107,99.885,475.0,0.000000,1.356913,-0.593750,8.ファイナル後半,92.0
2,019128GA010,2023,36,41.0,87.0,85.0,1.07,1.139097,1.0,ＴＰ３４２フタナシ,...,1.869305,0.29,0.761144,99.895,480.0,-0.163265,-0.521146,2.576923,8.ファイナル後半,85.0
3,019128GA010,2023,37,40.0,88.0,82.0,1.07,1.139097,1.0,ＴＰ３４２フタナシ,...,1.946419,0.29,0.789493,99.440,462.0,-0.024390,0.037037,-0.032258,8.ファイナル後半,92.0
4,019128GA010,2023,38,37.0,74.0,78.0,1.07,1.139097,1.0,ＴＰ３４２フタナシ,...,1.843305,0.29,0.748544,99.905,450.0,-0.075000,-0.035714,-0.033333,8.ファイナル後半,84.0
5,019128GA010,2023,39,40.0,84.0,82.0,1.07,1.139097,1.0,ＴＰ３４２フタナシ,...,1.936363,0.29,0.785726,99.955,464.0,0.081081,0.018519,0.057471,8.ファイナル後半,96.0
6,01912ECB010,2023,35,14.0,23.0,22.0,0.85,0.850000,1.0,ＴＰ３３２フタナシ,...,inf,0.29,0.000000,99.885,475.0,0.000000,0.000000,0.000000,8.ファイナル後半,29.0
7,01912ECB010,2023,35,14.0,23.0,22.0,0.85,1.057337,1.0,ＴＰ３３２フタナシ,...,10.286050,0.29,1.036061,99.885,475.0,0.000000,-0.312373,-0.590909,8.ファイナル後半,29.0
8,01912ECB010,2023,36,13.0,30.0,28.0,0.85,1.057337,1.0,ＴＰ３３２フタナシ,...,1.778120,0.29,0.680256,99.895,480.0,-0.071429,0.038348,2.444444,8.ファイナル後半,31.0
9,01912ECB010,2023,37,13.0,28.0,28.0,0.85,1.057337,1.0,ＴＰ３３２フタナシ,...,1.899045,0.29,0.717134,99.440,462.0,0.000000,-0.031250,0.000000,8.ファイナル後半,30.0


In [176]:
merged_data3['納入数/日量数']=merged_data3['納入かんばん数']/merged_data3['日量数（箱数）']
merged_data3['入庫数/納入数']=merged_data3['入庫数（箱）']/merged_data3['納入かんばん数']
merged_data3['出庫数/入庫数']=merged_data3['出庫数（箱）']/merged_data3['入庫数（箱）']
merged_data3['回収数/出庫数']=merged_data3['回収かんばん数']/merged_data3['出庫数（箱）']
merged_data3['回収数/納入数']=merged_data3['回収かんばん数']/merged_data3['納入かんばん数']
merged_data3['便Ave']=merged_data3['日量数（箱数）']/merged_data3['B']
merged_data3.head(5)

,品番,Year,Week_Number,在庫数（箱）,入庫数（箱）,出庫数（箱）,基準在庫日数,基準在庫日数（基準在庫枚数込み）,基準在庫枚数,箱種類,...,先週からの社内LT増加率,先週からの回収かんばん増加率,使用工程,納入かんばん数,納入数/日量数,入庫数/納入数,出庫数/入庫数,回収数/出庫数,回収数/納入数,便Ave
2,019128GA010,2023,36,41.0,87.0,85.0,1.07,1.139097,1.0,ＴＰ３４２フタナシ,...,-0.521146,2.576923,8.ファイナル後半,85.0,0.899090,1.023529,0.977011,1.094118,1.094118,47.270000
3,019128GA010,2023,37,40.0,88.0,82.0,1.07,1.139097,1.0,ＴＰ３４２フタナシ,...,0.037037,-0.032258,8.ファイナル後半,92.0,1.038609,0.956522,0.931818,1.097561,0.978261,44.290000
4,019128GA010,2023,38,37.0,74.0,78.0,1.07,1.139097,1.0,ＴＰ３４２フタナシ,...,-0.035714,-0.033333,8.ファイナル後半,84.0,0.970874,0.880952,1.054054,1.115385,1.035714,43.260000
5,019128GA010,2023,39,40.0,84.0,82.0,1.07,1.139097,1.0,ＴＰ３４２フタナシ,...,0.018519,0.057471,8.ファイナル後半,96.0,1.078167,0.875000,0.976190,1.121951,0.958333,44.520000
8,01912ECB010,2023,36,13.0,30.0,28.0,0.85,1.057337,1.0,ＴＰ３３２フタナシ,...,0.038348,2.444444,8.ファイナル後半,31.0,0.983711,0.967742,0.933333,1.107143,1.000000,15.756667


In [177]:
# Week_Numberが36のデータを削除
merged_data3 = merged_data3[merged_data3['Week_Number'] != 35]

# 集計したデータを新しいCSVファイルに保存します。
with open('data.csv', mode='w',newline='', encoding='shift_jis',errors='ignore') as f:
    merged_data3.to_csv(f)

# 確認のため、削除後のデータの最初の5行を表示
merged_data3.head(5)

,品番,Year,Week_Number,在庫数（箱）,入庫数（箱）,出庫数（箱）,基準在庫日数,基準在庫日数（基準在庫枚数込み）,基準在庫枚数,箱種類,...,先週からの社内LT増加率,先週からの回収かんばん増加率,使用工程,納入かんばん数,納入数/日量数,入庫数/納入数,出庫数/入庫数,回収数/出庫数,回収数/納入数,便Ave
2,019128GA010,2023,36,41.0,87.0,85.0,1.07,1.139097,1.0,ＴＰ３４２フタナシ,...,-0.521146,2.576923,8.ファイナル後半,85.0,0.899090,1.023529,0.977011,1.094118,1.094118,47.270000
3,019128GA010,2023,37,40.0,88.0,82.0,1.07,1.139097,1.0,ＴＰ３４２フタナシ,...,0.037037,-0.032258,8.ファイナル後半,92.0,1.038609,0.956522,0.931818,1.097561,0.978261,44.290000
4,019128GA010,2023,38,37.0,74.0,78.0,1.07,1.139097,1.0,ＴＰ３４２フタナシ,...,-0.035714,-0.033333,8.ファイナル後半,84.0,0.970874,0.880952,1.054054,1.115385,1.035714,43.260000
5,019128GA010,2023,39,40.0,84.0,82.0,1.07,1.139097,1.0,ＴＰ３４２フタナシ,...,0.018519,0.057471,8.ファイナル後半,96.0,1.078167,0.875000,0.976190,1.121951,0.958333,44.520000
8,01912ECB010,2023,36,13.0,30.0,28.0,0.85,1.057337,1.0,ＴＰ３３２フタナシ,...,0.038348,2.444444,8.ファイナル後半,31.0,0.983711,0.967742,0.933333,1.107143,1.000000,15.756667


In [57]:
step.head(300)

,品番,使用工程
0,34989ECE030,NaN
1,9036340A010,NaN
2,35145ECE020,2.MGケース
3,33490ECB010,7.ファイナル前半
4,33490ECE010,7.ファイナル前半
...,...,...
143,9036652A004,1.デフ・大物ギヤ・Brg
144,9036628A007,1.デフ・大物ギヤ・Brg
145,9036630A017,1.デフ・大物ギヤ・Brg
146,9036628A006,1.デフ・大物ギヤ・Brg
